In [ ]:
#Vincent Ning
#10010690


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from functools import partial
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/housing.csv')
median_num_bedrooms = df['total_bedrooms'].median()
df['total_bedrooms'].fillna(median_num_bedrooms, inplace=True)
missing_values_count = df.isnull().sum()
missing_values_count[0:10]

In [ ]:
warnings.filterwarnings('ignore')
X = df.iloc[:, 1:-1]
y = df.iloc[:, 9]
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1345375)

In [ ]:
op_obj_df = df.select_dtypes(include=['object']).copy()
replace_map = {'ocean_proximity': {'NEAR BAY': 1, '<1H OCEAN': 2, 'NEAR OCEAN': 3, 'ISLAND': 4,
                                  'INLAND':5}}
labels = op_obj_df['ocean_proximity'].astype('category').cat.categories.tolist()
replace_map_comp = {'ocean_proximity' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

op_obj_df_replace = op_obj_df.copy()
op_obj_df_replace.replace(replace_map_comp, inplace=True)

X_train_full['ocean_proximity'] = op_obj_df_replace['ocean_proximity']
X_test['ocean_proximity'] = op_obj_df_replace['ocean_proximity']

X_train_full.dtypes.sample(8)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1875, random_state=1345375)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

After the experiment, 8 layers with 100 layer size has the best network

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(100, activation="relu", input_shape=X_train.shape[1:]))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1))

model.compile(loss="mean_squared_error", optimizer="adam", metrics = ["mae"])

history = model.fit(X_train, y_train, epochs=30, validation_data = (X_val, y_val))

In [ ]:
preds = model.predict(X_test)
print(len(preds[ preds < 15000 ]))
print(len(preds[ preds > 50000 ]))

4119 test examples have predictions outside [15000, 500000]

In [ ]:
model.summary()

In [ ]:
model.evaluate(X_test, y_test)

The best MAE from lab 3 was 62325.8894. Our MAE from this lab is 39904.2647. It is a huge improvment of 22421.6247.


In [ ]:
from keras.datasets import cifar10
(X_train_1, y_train_1), (X_test_1, y_test_1) = cifar10.load_data()

In [ ]:
y_train_1 = keras.utils.to_categorical(y_train_1)
y_test_1 = keras.utils.to_categorical(y_test_1)
X_train_1 = X_train_1/255
X_test_1 = X_test_1/255

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, activation='relu', padding="SAME")
model_1 = keras.models.Sequential([    
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[32, 32, 3]),    
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),    
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_1.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
model_1.fit(X_train_1, y_train_1, epochs=15, validation_split=0.15)

In [ ]:
model_1.evaluate(X_test_1, y_test_1)

In [ ]:
preds_1 = model_1.predict(X_test_1)
print(preds_1[7])
print(np.argmax(preds_1[7]))
print(np.argmax(y_test_1[7]))
preds_1[7][np.argmax(preds_1[7])] - preds_1[7][ np.argmax(y_test_1[7])]


In [ ]:
margins = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
entries = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(preds_1)):
    if np.argmax(preds_1[i]) != np.argmax(y_test_1[i]):
        print(i)
        currentMargin = preds_1[i][np.argmax(preds_1[i])] - preds_1[i ][ np.argmax(y_test_1[i])]
        if currentMargin > margins[np.argmax(y_test_1[i])]:
            margins[np.argmax(y_test_1[i])] = currentMargin
            entries[np.argmax(y_test_1[i])] = i

In [ ]:
print(entries)
print(margins)

In [ ]:
index = 0
for i in entries:
    plt.title("Correct Label: " + str(np.argmax(y_test_1[i])) + "  Wrongly Predicted Label: " + str(np.argmax(preds_1[i])) + "  value of p_predicted-p_correct: " + str(margins[index]))
    plt.imshow(X_train_1[i])
    plt.show()
    index = index + 1
